In [32]:
from data.dataset.multiwoz import MultiWozDataset
from utils.args_parser import ArgsParser

import sys
sys.argv = ['test_args.py']


EVAL_SPLIT = "test"

opt = ArgsParser().parse()
opt.multiwoz_version = '2.1'
opt.use_action = True
opt.use_knowledge = True
opt.context_knowledge = True
opt.lexical = True

opt_delex = ArgsParser().parse()
opt_delex.multiwoz_version = '2.1'

# Load the dataset
data = MultiWozDataset(opt, split=EVAL_SPLIT, shuffle=False)
data_delex = MultiWozDataset(opt_delex, split=EVAL_SPLIT, shuffle=False)

loading cached data from resources/cached_data_lexical_test.pkl
loading cached data from resources/cached_data_delex_test.pkl


In [33]:
# structure the data
lex_dict = {}
delex_dict = {}
for d in data:
    lex_dict[d['name']] = d

for d in data_delex:
    delex_dict[d['name']] = d

In [34]:
generated_dict = {}
num_data = len(data)
for i, dial_name in enumerate(lex_dict):
    if EVAL_SPLIT == 'train' and i > 1000:
        break
    d = lex_dict[dial_name]
    d_delex = delex_dict[dial_name]
    print('{} [{}/{}] \r'.format(d['name'], i, num_data), end='')
    sys.stdout.flush()
    beliefs_raw = d['belief_raw']
    user = d['input_raw']
    system = d['target_raw']
    system_delex = d_delex['target_raw']
    target_response = system
    
    # get the gold action
    action = d['action_raw']
    target_action = []
    for turn_act in action:
        turn_action = []
        for act in turn_act:
            act_str = '{} {} {}'.format(act[0], act[1], act[2])
            turn_action.append(act_str)
        target_action.append(turn_action)
    
    # get the gold ds
    dialogue_aggregated_target_belief = []
    dialogue_target_belief = []
    for turn_belief in beliefs_raw:
        turn_belief_str = []
        for bs in turn_belief:
            domain, slot, value = bs
            if value in ['not mentioned', 'none']:
                continue
            bs_str = '{} {} {}'.format(domain.lower(), slot.lower(), value.lower())
            if bs_str not in dialogue_aggregated_target_belief:
                dialogue_aggregated_target_belief.append(bs_str)
            turn_belief_str.append(bs_str)
        dialogue_target_belief.append(turn_belief_str)

    generated_dict[d['name']] = {
        'target_belief': dialogue_aggregated_target_belief,
        'target_turn_belief': dialogue_target_belief,
        'generated_belief': dialogue_aggregated_target_belief,
        'generated_turn_belief': dialogue_target_belief,
        'target_response': target_response,
        'generated_response': target_response,
        'target_action': target_action,
        'generated_action': target_action,
        'target_user': user,
        # 'model_context': model_context
    }


In [35]:
generated_dict["PMUL3688.json"]

{'target_belief': ['attraction area centre',
  'train destination stansted airport',
  'train day friday',
  'train leaveat 08:45',
  'train departure cambridge',
  'train book people 3'],
 'target_turn_belief': [['attraction area centre'],
  ['attraction area centre'],
  ['attraction area centre',
   'train destination stansted airport',
   'train day friday'],
  ['attraction area centre',
   'train leaveat 08:45',
   'train destination stansted airport',
   'train day friday',
   'train departure cambridge'],
  ['attraction area centre',
   'train leaveat 08:45',
   'train destination stansted airport',
   'train day friday',
   'train departure cambridge',
   'train book people 3'],
  ['attraction area centre',
   'train leaveat 08:45',
   'train destination stansted airport',
   'train day friday',
   'train departure cambridge',
   'train book people 3']],
 'generated_belief': ['attraction area centre',
  'train destination stansted airport',
  'train day friday',
  'train leaveat

In [29]:
exp_name = "test_eval"
save_name = '{}_{}'.format(exp_name, EVAL_SPLIT)
save_name += '_nocarry'

In [31]:
import json
with open('{}.json'.format(save_name), 'wt') as f:
    json.dump(generated_dict, f)
